# --- Markdown Cell: Introduction ---
# Statistical Signal Detection

**Objective:** To apply established statistical methods to our clean dataset to identify drug-adverse event pairs that are reported more frequently than expected by chance.

**The Story:** Now that we have clean data, we can begin the hunt for signals. Our approach is to calculate a "signal score" for every single drug-event pair in the database. A high score suggests a potential safety issue that warrants a closer look. We will use a sophisticated method called Empirical-Bayes shrinkage to ensure our signals are statistically robust and not just random noise.


# Setup and Load Data


In [12]:

import pandas as pd
import numpy as np
from scipy.stats import gamma, chi2_contingency, chi2
from scipy.special import psi
import os

PROCESSED_DATA_PATH = '../data/processed/'
TARGET_QUARTER_DIR = 'faers_ascii_2024q4'
ANALYSIS_READY_FILE = f'faers_{TARGET_QUARTER_DIR}_analysis_ready.parquet'

df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, ANALYSIS_READY_FILE))
print("Clean dataset loaded successfully.")



Clean dataset loaded successfully.



# Build Contingency Table and Calculate Signal Scores


In [13]:

# --- Explode data and get counts ---
df_pairs = df.explode('drugs').explode('reactions').dropna(subset=['drugs', 'reactions'])
pair_counts = df_pairs.groupby(['drugs', 'reactions']).size().reset_index(name='a')
drug_counts = df_pairs.groupby('drugs')['safetyreportid'].nunique().rename('N_drug')
event_counts = df_pairs.groupby('reactions')['safetyreportid'].nunique().rename('N_event')
N = df['safetyreportid'].nunique()

# --- Create the full stats table ---
df_stats = pair_counts.merge(drug_counts, on='drugs').merge(event_counts, on='reactions')
df_stats['b'] = df_stats['N_drug'] - df_stats['a']
df_stats['c'] = df_stats['N_event'] - df_stats['a']
df_stats['d'] = N - (df_stats['a'] + df_stats['b'] + df_stats['c'])
df_stats['E'] = (df_stats['N_drug'] * df_stats['N_event']) / N

# --- Calculate EBGM and EB05 (Robust Scores) ---
rr_observed = df_stats.loc[df_stats['a'] > 0, 'a'] / df_stats.loc[df_stats['a'] > 0, 'E']
alpha_prior = (rr_observed.mean() ** 2) / rr_observed.var()
beta_prior = rr_observed.mean() / rr_observed.var()

df_stats['alpha_post'] = alpha_prior + df_stats['a']
df_stats['beta_post'] = beta_prior + df_stats['E']

df_stats['EBGM'] = np.exp(psi(df_stats['alpha_post']) - np.log(df_stats['beta_post']))
df_stats['EB05'] = gamma.ppf(0.05, a=df_stats['alpha_post'], scale=1.0 / df_stats['beta_post'])
df_stats['EB_Signal'] = df_stats['EB05'] >= 2


# Save Final Results


In [14]:

# Also calculate p-value for the volcano plot in the next notebook
df_stats['chi2'] = df_stats.apply(lambda row: chi2_contingency(np.array([[row['a'], row['b']], [row['c'], row['d']]]))[0], axis=1)
df_stats['p_value'] = chi2.sf(df_stats['chi2'], 1)

RESULTS_OUTPUT = 'faers_full_results.parquet'
df_stats.to_parquet(os.path.join(PROCESSED_DATA_PATH, RESULTS_OUTPUT))

print(f"\n✅ Success! Full statistical results saved to {RESULTS_OUTPUT}")

# --- Markdown Cell: Conclusion ---
# # Conclusion
#
# We have successfully calculated robust signal scores (EBGM and EB05) for every drug-event pair. The `EB05` score gives us a high-confidence measure to rank and prioritize these signals.
#
# **Our final output is a comprehensive table containing these prioritized signals, which is now ready for visualization and deep-dive analysis.**
#
# Here is a preview of our top 10 most robust signals:
print("\n--- Top 10 Signals by EB05 Score ---")
print(df_stats[df_stats['EB_Signal']].sort_values('EB05', ascending=False).head(10)[['drugs', 'reactions', 'a', 'E', 'EBGM', 'EB05']].round(2))


✅ Success! Full statistical results saved to faers_full_results.parquet

--- Top 10 Signals by EB05 Score ---
       drugs reactions  a    E      EBGM     EB05
410147   583  23427330  1  0.0  84058.97  7687.96
410846    96  24448813  1  0.0  84058.97  7687.96
410847    97  19965923  1  0.0  84058.97  7687.96
410125   517  24282354  1  0.0  84058.97  7687.96
410142    55  24049810  1  0.0  84058.97  7687.96
410843    91  24591193  1  0.0  84058.97  7687.96
410120   503  23087756  1  0.0  84058.97  7687.96
410119    50  23679847  1  0.0  84058.97  7687.96
409693    47  24658635  1  0.0  84058.97  7687.96
409690    45  23237653  1  0.0  84058.97  7687.96
